In [3]:
import pandas as pd

data_file_path = "./datasets/generated/cleaned_aggregated_data.json"

df = pd.read_json(data_file_path)

df.head()

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,10,Y,0.294118,189.512143,84.572143,42.982857,19.696786,7.073929,44.290357,0.408655,0.717564,0.306577,0.500000,0
1,WAS,9,N,0.470588,189.708943,82.593336,36.207412,20.147909,8.820809,38.090105,0.402735,0.800336,0.276818,0.500000,2
2,WAS,8,N,0.529412,239.624444,95.275556,50.680741,25.660000,10.042222,44.559259,0.434105,0.760861,0.303131,0.529412,2
3,WAS,7,Y,0.470588,216.456929,88.321161,42.381086,23.959738,8.204120,39.441011,0.422553,0.743379,0.337256,0.470588,2
4,WAS,6,N,0.500000,193.306310,92.482727,39.100875,21.625518,11.273146,39.394288,0.429846,0.706172,0.334885,0.437500,1


## Decision Tree Classifier

In [54]:
#load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [103]:
#features
features = list(df.columns)
features.remove('playoff')
features.remove('tmID')
target = 'playoff'
X = df[features]
y = df[target]

Random split of dataset

In [95]:
# Split dataset into training set and test set - random split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

Split dataset such as: Train data - latest 7 years, Test data - first 3 years

In [104]:
#Split dataset in such way that we use latest years for training and older years for testing
X_train=df[df["year"]>3][features]
X_test=df[df["year"]<=3][features]
y_train=df[df["year"]>3]["playoff"]
y_test=df[df["year"]<=3]["playoff"]

We normalize the feature values so we have more training stability

In [105]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.

Import models

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

Create model objects

In [107]:
models = {}

#Logistic Regression
models['Logistic Regression'] = LogisticRegression()

#Support Vector Machines
models['Support Vector Machines'] = LinearSVC()

#Decision Trees
models['Decision Trees'] = DecisionTreeClassifier()

#Random Forest
models['Random Forest'] = RandomForestClassifier()

#Naive Bayes
models['Naive Bayes'] = GaussianNB()

#K-Nearest Neighbors
models['K-Nearest Neighbor'] = KNeighborsClassifier()

Train, predict and calculate scores

In [108]:
accuracy, precision, recall = {}, {}, {}

for model in models.keys():
    
    # Fit the classifier
    models[model].fit(X_train, y_train)
    
    # Make predictions
    y_pred = models[model].predict(X_test)
    
    # Calculate metrics
    accuracy[model] = metrics.accuracy_score(y_test, y_pred)
    precision[model] = metrics.precision_score(y_test, y_pred, pos_label='Y')
    recall[model] = metrics.recall_score(y_test, y_pred, pos_label='Y')

d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Anaconda3\lib\site-packages\s

Results for random split

In [93]:
df_results = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_results['Accuracy'] = accuracy.values()
df_results['Precision'] = precision.values()
df_results['Recall'] = recall.values()

df_results

,Accuracy,Precision,Recall
Logistic Regression,0.578947,0.571429,0.631579
Support Vector Machines,0.605263,0.600000,0.631579
Decision Trees,0.605263,0.600000,0.631579
Random Forest,0.631579,0.586207,0.894737
Naive Bayes,0.657895,0.650000,0.684211
K-Nearest Neighbor,0.631579,0.631579,0.631579


Results for split according to years

In [109]:
df_results = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_results['Accuracy'] = accuracy.values()
df_results['Precision'] = precision.values()
df_results['Recall'] = recall.values()

df_results

,Accuracy,Precision,Recall
Logistic Regression,0.53125,0.521739,0.7500
Support Vector Machines,0.56250,0.545455,0.7500
Decision Trees,0.62500,0.600000,0.7500
Random Forest,0.53125,0.523810,0.6875
Naive Bayes,0.62500,0.611111,0.6875
K-Nearest Neighbor,0.62500,0.611111,0.6875
